In [ ]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,StandardScaler,MinMaxScaler
from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
from datetime import datetime
from sksurv.svm import FastSurvivalSVM
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import ShuffleSplit,GridSearchCV
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import recall_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.metrics import precision_score, auc, roc_auc_score
from sklearn.metrics import roc_curve, precision_recall_curve 
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from lifelines.utils import median_survival_times
from lifelines.utils import survival_table_from_events
from lifelines import KaplanMeierFitter 
import time
from lifelines import CoxPHFitter
from sklearn.linear_model import Lasso,LassoLarsIC
from sksurv.linear_model import CoxPHSurvivalAnalysis,CoxnetSurvivalAnalysis
from sklearn.pipeline import make_pipeline
from sksurv.metrics import cumulative_dynamic_auc
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
import torchtuples as tt
from scipy.stats import randint 
from imblearn.over_sampling import SMOTE
from collections import Counter
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import shap

In [ ]:
raw_imputed.to_csv('5.Prostate.csv',index=False)

In [ ]:
from ReusableUtils import ReusableUtils
utils = ReusableUtils()
utils.setNotebookConfigParams()

In [ ]:
pd.set_option('display.max_info_columns',500)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',10)

In [ ]:
df_prostate = pd.read_csv('Prostate.csv')

In [ ]:
df_prostate.columns

In [ ]:
df_prostate.drop(columns=['pad_flag','ckd_cd','short_desc','dialysis_flag'],inplace=True)

In [ ]:
col_names = ['time', 'status', 'sex','diagnosis_Age','bmi','central_obesity_flag', 'smoke_status_cd','alcohol_status_cd', 
             'dm_period','family_dm_flag', 'chd_flag', 'stroke_flag', 'ht_flag',
             'insulin_flag', 'anti_htn_flag', 'anti_lipid_flag','diabetic_drug', 
             'hba1c', 'fasting_gc', 'total_clt', 'ldl_c', 'hdl_c','triglyceride', 'serum_k', 'creatinine']
df_prostate = df_prostate[col_names]

In [ ]:
df_prostate_dummy = pd.get_dummies(df_prostate,drop_first=True)# drop_first=True表示丢弃第一个类别，使之信息不冗余
display(df_prostate_dummy)

In [ ]:
# 缺失数据填补
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)# KNN全称: k nearest neighbor（K近邻算法）
imputer.fit(df_prostate_dummy)# 拟合
raw_imputed = imputer.transform(df_prostate_dummy)# 变换
# 将填补后的协变量（数据格式）转为dataframe格式：
raw_imputed = pd.DataFrame(columns=df_prostate_dummy.columns, data=raw_imputed)
display(raw_imputed)

In [ ]:
data = df_prostate

In [ ]:
df_prostate.columns

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency
# Assuming your data is stored in a pandas DataFrame called 'data'

# Step 1: Perform train-test split (70% train, 30% test)
train_ratio = 0.7
train_size = int(train_ratio * len(data))
train_data = data.sample(train_size, random_state=42)
test_data = data.drop(train_data.index)

# Step 2: Calculate statistics for each feature
result_data = pd.DataFrame(columns=['Feature', 'Train Value', 'Test Value'])
target = 'status'

for column in data.columns[1:-1]:
    # For numerical features, calculate mean and standard deviation
    if np.issubdtype(data[column].dtype, np.number):
        train_mean = train_data[column].mean()
        train_std = train_data[column].std()
        test_mean = test_data[column].mean()
        test_std = test_data[column].std()
        result_data = result_data.append({
            'Feature': column,
            'Train Value': f'Mean ± Std: {train_mean:.2f} ± {train_std:.2f}',
            'Test Value': f'Mean ± Std: {test_mean:.2f} ± {test_std:.2f}'
        }, ignore_index=True)
    # For categorical features, calculate value counts as percentages
    else:
        train_counts = train_data[column].value_counts(normalize=True) * 100
        test_counts = test_data[column].value_counts(normalize=True) * 100
        train_values_str = "\n ".join([f'{value}: {train_data[column].value_counts()[value]} ({train_percent:.2f}%)' for value, train_percent in train_counts.iteritems()])
        test_values_str = "\n ".join([f'{value}: {test_data[column].value_counts().get(value, 0)} ({test_percent:.2f}%)' for value, test_percent in test_counts.iteritems()])
        result_data = result_data.append({
            'Feature': column,
            'Train Value': train_values_str,
            'Test Value': test_values_str
        }, ignore_index=True)
        
pd.set_option('display.max_colwidth', -1)

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 自动检查并标签编码所有的类别特征
data_encoded = data.copy()
cols = data_encoded.columns[1:-1]
for column in cols:
    if data_encoded[column].dtype == 'object':
        label_encoder = LabelEncoder()
        data_encoded[column] = label_encoder.fit_transform(data_encoded[column])

# 计算所有特征与目标变量之间的Pearson相关系数
correlations = data_encoded[cols].corrwith(data_encoded[target], method='spearman').rename("P").abs().reset_index(drop=True)

In [ ]:
def corr_plot(data,need_corr=True,name='',title=''):
	plt.subplots(figsize = (30,20))
	sns.heatmap(data.corr(method='pearson') if need_corr else data, 
				annot=True,
				cmap = 'RdBu',#色盘，可以理解为配色主题(在这里用Palette参数会报错)
				linewidths = 0.1,#热力矩阵之间的线宽
				linecolor='white',
				vmax = 0.9,#vmax,vmin, 图例(热力bar)中最大值和最小值的显示值，没有该参数时默认不显示
				square=False
				)
# 	plt.title(title if name==None else title+name,fontsize = 20)
	plt.tight_layout()
	plt.savefig('Kid_correlation.pdf')
	plt.show()


# data是dataframe类型
corr_plot(data_encoded[data_encoded.columns[3:]],title = 'Feature Correlation')

## Machine Learning

In [ ]:
X = df_prostate.drop(columns=['time','status'])
X_dummy = pd.get_dummies(X,drop_first=True)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(X_dummy)
X_imputed = imputer.transform(X_dummy)
X_imputed = pd.DataFrame(columns=X_dummy.columns, data=X_imputed)

In [ ]:
from sklearn.model_selection import train_test_split
y = raw_imputed.loc[:,['status','time']]
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=2022)

In [ ]:
from sksurv.util import Surv
# 训练集
y_train_ = Surv.from_dataframe(
    event='event', 
    time='time', 
    data=y_train.rename(columns={'status':'event','malignant_surviving_month':'time'}))
y_train_
# 测试集
y_test_ = Surv.from_dataframe(
    event='event', 
    time='time', 
    data=y_test.rename(columns={'status':'event','malignant_surviving_month':'time'}))

## CoxPH

In [ ]:
features = raw_imputed.columns[2:]
summary_results = pd.DataFrame()
# 循环对每个类别特征进行单因素COX回归分析
for feature in features:
    
    cph = CoxPHFitter()

    # 进行单因素COX回归分析
    cph.fit(raw_imputed[[feature,'time', 'status']], duration_col='time', event_col='status')

    # 提取结果并添加到汇总表
    result = cph.summary.reset_index()
    result['Feature'] = feature
    summary_results = pd.concat([summary_results, result])

# 重置索引并重命名列名
summary_results.reset_index(drop=True, inplace=True)
# summary_results.columns = ['Feature', 'Hazard Ratio', '95% CI (Lower)', '95% CI (Upper)', 'P-value']

# 输出汇总表
results1 = summary_results[['covariate','exp(coef)','exp(coef) lower 95%', 'exp(coef) upper 95%', 'p']].copy()
results1.columns = ['Variables','HR2','exp(coef) lower 95%', 'exp(coef) upper 95%', 'P_value']
results1['95% CI'] = results1['exp(coef) lower 95%'].map(lambda x:f'{x:.2f}') +', '+ results1['exp(coef) upper 95%'].map(lambda x:f'{x:.2f}')
results1.drop(columns=['exp(coef) lower 95%', 'exp(coef) upper 95%'],inplace=True)

In [ ]:
# 创建CoxPHFitter对象
cph = CoxPHFitter()

# 进行单因素COX回归分析
cph.fit(raw_imputed[raw_imputed.columns[0:]], duration_col='time', event_col='status')

# 提取结果
results2 = cph.summary.reset_index()[['covariate','exp(coef)','exp(coef) lower 95%', 'exp(coef) upper 95%', 'p']].rename(columns={'p':'P_value'})
results2.columns = ['Variables','HR2','exp(coef) lower 95%', 'exp(coef) upper 95%', 'P_value']
results2['95% CI'] = results2['exp(coef) lower 95%'].map(lambda x:f'{x:.2f}') +', '+ results2['exp(coef) upper 95%'].map(lambda x:f'{x:.2f}')
results2.drop(columns=['exp(coef) lower 95%', 'exp(coef) upper 95%'],inplace=True)
results_cox = pd.merge(results1,results2,on='Variables',suffixes=['_Univariate Cox','_Multivariate Cox'])

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

cox = CoxPHSurvivalAnalysis()
cox.fit(X_train, y_train_)# 模型拟合

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = cox.predict(X_train)
result = concordance_index_censored(y_train_['event'], y_train_['time'], y_pred)
display(result)

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = cox.predict(X_test)
result = concordance_index_censored(y_test_['event'], y_test_['time'], y_pred)
display(result)

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc

times = np.arange( y_test_['time'].min(),y_test_['time'].max() )
chfs = cox.predict_cumulative_hazard_function(X_test)
#preds = np.asarray([[fn(t) for t in times] for fn in chfs])
preds = cox.predict(X_test)

In [ ]:
## tcauc cph

import matplotlib.pyplot as plt
preds = cox.predict(X_test)
aucs_cph, mean_auc_cph = cumulative_dynamic_auc(y_train_, y_test_, preds, times)
fig, ax = plt.subplots(figsize=(8, 4))
# aucs curve
plt.plot(times,aucs_cph)
# mean auc
plt.axhline(y=mean_auc_cph, color='black', ls ='--')
ax.annotate('mean auc CoxPH={:.3f}'.format(mean_auc_cph), xy=(100,mean_auc_cph), xytext=(200,mean_auc_cph+0.01), fontsize=10)

plt.xlabel('time(month)')
plt.ylabel('auc')
plt.show()

In [ ]:
exp_cph = shap.Explainer(cox.predict,X_train)
shap_values_cph = exp_cph(X_test)
#shap.plots.beeswarm(shap_values,max_display=44)
shap.summary_plot(shap_values_cph,X_test,max_display=44,plot_size=(12,20),show=False)
plt.savefig('coxph_shap_pro.pdf',bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
# help(cph.plot)
fig,ax = plt.subplots(figsize=(4,6))#dpi=120
#plt.title('Forest graph -- ')
cph.plot(ax=ax,hazard_ratios=True)# 森林图
#cph.plot(ax=ax)
ax.grid(color='grey',ls='--')#网格

#plt.tight_layout()
plt.savefig('Forest_graph_Col.pdf',bbox_inches='tight')

## Tree

In [ ]:
from sksurv.tree import SurvivalTree
tree = SurvivalTree(max_depth=20, min_samples_leaf=5, min_samples_split=3)

#time setting
lower,upper = np.percentile(y['time'],[10,90])
times = np.arange(lower,upper + 1)
from sksurv.tree import SurvivalTree
tree = SurvivalTree(max_depth=2, min_samples_leaf=2, min_samples_split=3)
#gcv

from sklearn.model_selection import GridSearchCV, KFold

# 实例化一个网格搜索器
# # 方式1：
parameters = {
    'max_depth':[2, 10, 20],
    'min_samples_split':[3,6,9], 
    'min_samples_leaf':[2,3,4,5],
}# 字典，超参数及其可能的取值
gscv = GridSearchCV(tree, parameters)

In [ ]:
print("start training...")
start_time = time.time()
#Search
gscv.fit(X_train, y_train_)# 在训练集上寻优
print(f'---{round(time.time()-start_time,3)}seconds ---')
display(gscv.best_params_)

In [ ]:
tree = SurvivalTree(max_depth=10, min_samples_leaf=2, min_samples_split=9)
tree.fit(X_train, y_train_)
from sksurv.metrics import concordance_index_censored

y_pred = tree.predict(X_train)
result = concordance_index_censored(y_train_['event'], y_train_['time'], y_pred)
display(result)


In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = tree.predict(X_test)
result = concordance_index_censored(y_test_['event'], y_test_['time'], y_pred)
display(result)

In [ ]:
exp = shap.Explainer(tree.predict,X_train)
shap_values = exp(X_test)
shap.plots.beeswarm(shap_values,max_display=44)

In [ ]:
exp_tree = shap.Explainer(tree.predict,X_train)
shap_values_tree = exp_tree(X_test)
shap.summary_plot(shap_values_tree,X_test,max_display=44,plot_size=(12,20),show=False)
plt.savefig('tree_shap_prostate.pdf',bbox_inches='tight')

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc

times = np.arange( y_test_['time'].min(),y_test_['time'].max() )
treesurv = tree.predict_cumulative_hazard_function(X_test)
preds = np.asarray([[fn(t) for t in times] for fn in treesurv])
aucs_tree, mean_auc_tree = cumulative_dynamic_auc(y_train_, y_test_, preds, times)
aucs_tree = np.nan_to_num(aucs_tree,nan=0)
mean_auc_tree = np.mean(aucs_tree)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
# aucs curve
plt.plot(times,aucs_tree)
# mean auc
plt.axhline(y=mean_auc_tree, color='black', ls ='--')
ax.annotate('Decision Tree mean auc={:.3f}'.format(mean_auc_tree), xy=(100,mean_auc_tree), xytext=(150,mean_auc_tree+0.01), fontsize=10)

plt.xlabel('time(month)')
plt.ylabel('auc')
plt.show()

## RSF

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=20)

In [ ]:
print("start training...")
start_time = time.time()
rsf.fit(X_train, y_train_)
print(f'---{round(time.time()-start_time,3)}seconds ---')

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = rsf.predict(X_train)
result = concordance_index_censored(y_train_['event'], y_train_['time'], y_pred)
display(result)

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = rsf.predict(X_test)
result = concordance_index_censored(y_test_['event'], y_test_['time'], y_pred)
display(result) 

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc

times = np.arange( y_test_['time'].min(),y_test_['time'].max() )
rsfsurv = rsf.predict_cumulative_hazard_function(X_test)
preds = np.asarray([[fn(t) for t in times] for fn in rsfsurv])
aucs_rsf, mean_auc_rsf = cumulative_dynamic_auc(y_train_, y_test_, preds, times)
aucs_rsf = np.nan_to_num(aucs_rsf,nan=0)
mean_auc_rsf = np.mean(aucs_rsf)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
# aucs curve
plt.plot(times,aucs_rsf)
# mean auc
plt.axhline(y=mean_auc_rsf, color='black', ls ='--')
ax.annotate('RSF mean auc={:.3f}'.format(mean_auc_rsf), xy=(100,mean_auc_rsf), xytext=(150,mean_auc_rsf+0.01), fontsize=10)

plt.xlabel('time(month)')
plt.ylabel('auc')
plt.show()


In [ ]:
exp_rsf = shap.Explainer(rsf.predict,X_train)
shap_values_rsf = exp_rsf(X_test)
#shap.plots.beeswarm(shap_values_rsf,max_display=44)
shap.summary_plot(shap_values_rsf,X_test,max_display=44,plot_size=(12,20),show=False)
plt.savefig('rsf_shap_pro.pdf',bbox_inches='tight')

## Coxboost

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
coxboost =  GradientBoostingSurvivalAnalysis(loss="coxph")

In [ ]:
coxboost.fit(X_train, y_train_)

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = coxboost.predict(X_train)
result = concordance_index_censored(y_train_['event'], y_train_['time'], y_pred)
display(result)

In [ ]:
from sksurv.metrics import concordance_index_censored

y_pred = coxboost.predict(X_test)
result = concordance_index_censored(y_test_['event'], y_test_['time'], y_pred)
display(result)

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc

times = np.arange( y_test_['time'].min(),y_test_['time'].max() )
coxboostsurv = coxboost.predict_cumulative_hazard_function(X_test)
#preds = np.asarray([[fn(t) for t in times] for fn in coxboostsurv])
preds = y_pred
aucs_coxboost, mean_auc_coxboost = cumulative_dynamic_auc(y_train_, y_test_, preds, times)
aucs_coxboost = np.nan_to_num(aucs_coxboost,nan=0)
mean_auc_coxboost = np.mean(aucs_coxboost)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
# aucs curve
plt.plot(times,aucs_coxboost)
# mean auc
plt.axhline(y=mean_auc_coxboost, color='black', ls ='--')
ax.annotate('Coxboost mean auc={:.3f}'.format(mean_auc_coxboost), xy=(100,mean_auc_coxboost), xytext=(150,mean_auc_coxboost+0.01), fontsize=10)

plt.xlabel('time(month)')
plt.ylabel('auc')
plt.show()

In [ ]:
exp = shap.Explainer(coxboost.predict,X_train)
shap_values = exp(X_test)
shap.plots.beeswarm(shap_values,max_display=44)

In [ ]:
exp_coxboost = shap.Explainer(coxboost.predict,X_train)
shap_values_coxboost = exp_coxboost(X_test)
#shap.plots.beeswarm(shap_values_rsf,max_display=44)
shap.summary_plot(shap_values_coxboost,X_test,max_display=44,plot_size=(12,20),show=False)
plt.savefig('coxboost_shap_pro.pdf',bbox_inches='tight')

## Coxnet

In [ ]:
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio = 1.0,alpha_min_ratio = 0.01,fit_baseline_model=True)
cox_lasso.fit(X_train, y_train_)
cindex_train = cox_lasso.score(X_train, y_train_)
print(round(cindex_train,10))

In [ ]:
cindex_test = cox_lasso.score(X_test,y_test_)
print(round(cindex_test,10))

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc

times = np.arange( y_test_['time'].min(),y_test_['time'].max() )
cox_lassosurv = cox_lasso.predict_cumulative_hazard_function(X_test)
#preds = np.asarray([[fn(t) for t in times] for fn in cox_lassosurv])
preds = cox_lasso.predict(X_test)
aucs_coxnet, mean_auc_coxnet = cumulative_dynamic_auc(y_train_, y_test_, preds, times)
aucs_coxnet = np.nan_to_num(aucs_coxnet,nan=0)
mean_auc_coxnet = np.mean(aucs_coxnet)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
# aucs curve
plt.plot(times,aucs_coxnet)
# mean auc
plt.axhline(y=mean_auc_coxnet, color='black')
ax.annotate('Coxnet mean auc={:.3f}'.format(mean_auc_coxnet), xy=(100,mean_auc_coxnet), xytext=(150,mean_auc_coxnet+0.01), fontsize=10)
#ax.annotate('Decision Tree mean auc={:.3f}'.format(mean_auc_tree), xy=(100,mean_auc_tree), xytext=(150,mean_auc_tree+0.01), fontsize=10)
plt.xlabel('time(month)')
plt.ylabel('auc')
plt.show()

In [ ]:
exp = shap.Explainer(cox_lasso.predict,X_train)
shap_values = exp(X_test)
shap.plots.beeswarm(shap_values,max_display=44)

In [ ]:
exp_coxnet = shap.Explainer(cox_lasso.predict,X_train)
shap_values_coxnet = exp_coxnet(X_test)
#shap.plots.beeswarm(shap_values_rsf,max_display=44)
shap.summary_plot(shap_values_coxnet,X_test,max_display=44,plot_size=(12,20),show=False)
plt.savefig('coxnet_shap_pro.pdf',bbox_inches='tight')

## tc-auc

In [ ]:
plt.ylim(0.38,1)


plt.plot(times, aucs_coxboost, "-", label=f"CoxBoost (mean AUC = {mean_auc_coxboost:.3f})",color = "#551F33")
plt.plot(times, aucs_coxnet, "-", label=f"CoxNet (mean AUC = {mean_auc_coxnet:.3f})",color = "#F5E4C8")
plt.plot(times, aucs_rsf, "-", label=f"RSF (mean AUC = {mean_auc_rsf:.3f})",color = "#BD4146")
plt.plot(times, aucs_cph, "-", label=f"CoxPH (mean AUC = {mean_auc_cph:.3f})",color = "#ECC68C")
plt.plot(times, aucs_tree, "-", label=f"Survival Tree (mean AUC = {mean_auc_tree:.3f})",color = "#CBBBC1")


plt.xlabel("Time(Months)",fontsize=12)
plt.ylabel("Time-dependent AUC",fontsize=12)
plt.legend(loc="lower right",fontsize=8)
plt.grid(True)
plt.savefig('Time_dependent_auc_Prostate.pdf',bbox_inches='tight')